# 2022 Flatiron Machine Learning x Science Summer School

## Step 5: Train DSN with $L_1$ regularization on latent features

### Step 5.1: Check $a_1$ and $a_2$ parameters

In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
import joblib

import torch
import wandb

from srnet import SRNet, SRData
import srnet_utils as ut

In [2]:
# set wandb project
wandb_project = "51-a1-a2-study-F00"

In [3]:
# define hyperparameters
# hyperparams = {
#     "arch": {
#         "in_size": train_data.in_data.shape[1],
#         "out_size": train_data.target_data.shape[1],
#         "hid_num": (2,0),
#         "hid_size": 32, 
#         "hid_type": ("DSN", "MLP"),
#         "lat_size": 16,
#         },
#     "epochs": 10000,
#     "runtime": None,
#     "batch_size": 64,
#     "lr": 1e-4,
#     "wd": 1e-4,
#     "l1": 0.0,
#     "a1": 0.0,
#     "a2": 0.0,
#     "shuffle": True,
# }

In [4]:
# define hyperparameter study
hp_study = {
    "method": "grid", # random, bayesian
    #"metric": {
    #    "name": "val_loss",
    #    "goal": "minimize",
    #},
    "parameters": {
        "a1": {
            "values": [0.0, 1e-5, 1e-3, 1e-1]
        },
        "a2": {
            "values": [0.0, 1e-5, 1e-3, 1e-1]
        }
    }
}

In [5]:
# create sweep
sweep_id = wandb.sweep(hp_study, project=wandb_project)

Create sweep with ID: 2rtgnxtt
Sweep URL: https://wandb.ai/fabxy/51-a1-a2-study-F00/sweeps/2rtgnxtt


In [58]:
# download data from wandb
file_ext = ".pkl"

api = wandb.Api()

runs = api.runs(wandb_project)
for run in runs:
    for f in run.files():
        if f.name[-len(file_ext):] == file_ext and not os.path.isfile(f.name):
            print(f"Downloading {os.path.basename(f.name)}.")
            run.file(f.name).download()

What was actually done in the work? Let's read the repo.

In [ ]:
model = nn.Sequential(
    nn.Linear(n_in, hidden), nn.ReLU(),
    nn.Linear(hidden, hidden), nn.ReLU(),
    nn.Linear(hidden, hidden), nn.ReLU(),
    nn.Linear(hidden, n_out))

In [ ]:
from sjnn.torch import SparseJacobianNN as SJNN
# from sjnn.haiku import SparseJacobianNN as SJNN
model = SJNN(
    n_in=n_in, n_out=n_out,
    hidden=hidden, nlayers=2, act=F.relu)

In [ ]:
loss += model.sparsifying_loss(x)

In [ ]:
a1, a2: L1 reg and sparsity reg

In [ ]:
approx_l0, approx_max

In [ ]:
alpha?

In [ ]:
weights:
    self.w = nn.ParameterList(
            [init_weights_3d(n_out, n_in, hidden)]
            + [init_weights_3d(n_out, hidden, hidden) for i in range(nlayers)]
            + [init_weights_3d(n_out, hidden, 1)]
        )

In [ ]:
biases:
    self.b = nn.ParameterList(
            [init_weights_2d(n_out, hidden)]
            + [init_weights_2d(n_out, hidden) for i in range(nlayers)]
            + [init_weights_2d(n_out, 1)]
        )

In [ ]:
forward:
    get_masked_input        
    n_layer x: apply_layer
    out: apply_layer

In [ ]:
get_masked_input(self, x: TensorType["batch", "input"]) -> TensorType["batch", "output", "input"]:
    tiled = E.repeat(x, "batch n_in -> batch n_out n_in", n_out=self.n_out)
    return tiled * self.alpha[None]

# E is from einops
# alpha is n_out x n_in
# 

In [ ]:
importances = self.alpha  # [n_out, n_in]
few_latents = self.l0_func(importances, dim=1).sum()
few_dependencies = self.max_func(self.l0_func(importances, dim=0), dim=0)
return self.a1 * few_latents + self.a2 * few_dependencies

In [ ]:
n_out x n_in    # lat_feat x in_feat

abs(alpha).sum() across n_in # lat_feat
abs(alpha).sum() # 1

abs(alpha).sum() across n_out # in_feat                    TODO: is that correct? how do the dimensions work here?
self.max_func(self.l0_func(importances, dim=0), dim=0) 

dim=0 to dim=1 -> pull request?